# Multi-Branch Interactions & Patterns

This notebook demonstrates how to orchestrate multiple AI agents working together, showcasing the power of lionagi's branch system for complex AI workflows.

**What you'll learn:**
- Creating separate AI agents with distinct personalities and roles
- Passing context between different branches for collaboration
- Implementing feedback loops for iterative improvement
- Persisting and reloading conversation state

In [1]:
from lionagi import Branch
from IPython.display import Markdown

In [2]:
sys_comedian = "As a comedian, you are sarcastically funny"

prompt1 = "short joke: a blue whale and a big shark meet at the bar and start dancing"
prompt2 = "continue the joke: and then they stopped"

In [3]:
# start a LLM session and have consecutive conversation with it
comedian = Branch(system=sys_comedian)

joke1 = await comedian.communicate(prompt1)
joke2 = await comedian.communicate(prompt2)

In [4]:
Markdown(joke1)

A blue whale and a big shark walk into a bar, spot each other, and start dancing. Turns out, one's just there to whale out, and the other's just looking to fin-ish the night on a high note!

In [5]:
Markdown(joke2)

And then they stopped... because even a giant whale and a massive shark knew it’s hard to keep dancing when the music’s just a little too *fin*-tastic.

**Introducing Multi-Agent Collaboration:**

Now we'll create a second AI agent with a different role and personality. This demonstrates a common pattern in AI applications: having specialized agents that excel at different tasks.

**The Multi-Agent Pattern:**
- **Comedian**: Creative, humorous, focused on entertainment
- **Critic**: Analytical, logical, focused on evaluation and improvement

This separation of concerns allows each agent to specialize in what they do best, leading to higher quality results through collaboration.

In [6]:
# some messages for the critic
sys_critic = "you are a respected commentator, you are artistically logical"

prompt3 = "short comments, what do you think about the first joke?"
prompt4 = "provide a concise artistic critique on both jokes, and rate from 1-10"

In [7]:
# create a session with the critic and generate some comments in a multi-turn conversation
critic = Branch(system=sys_critic)

# provide context for the critic, context needs to be string or string mappings/iterables
context = {
    "joke1": {"prompt": prompt1, "response": joke1},
    "joke2": {"prompt": prompt2, "response": joke2},
}

# you only need to provide same context once in a conversation
comment1 = await critic.communicate(prompt3, context=context)
comment2 = await critic.communicate(prompt4)

In [8]:
Markdown(comment1)

The first joke charmingly plays with oceanic imagery and puns, giving the animals human-like social traits. It's a whimsical, light-hearted setup that invites a fun visual—two giant sea creatures dancing in a bar—highlighting humor through clever wordplay. A delightful way to animate the marine world with playful humor!

In [9]:
Markdown(comment2)

The first joke creatively merges marine life with barroom social scene, using puns like "whale out" and "fin-ish" to evoke humor through wordplay. Its vivid imagery and light tone make it engaging.  
**Rating: 7/10**

The second joke extends the pun theme, ending with a clever twist on the difficulty of dancing to "fintastic" music. It employs a nice callback while maintaining playful language.  
**Rating: 6.5/10**

**Implementing Feedback Loops:**

One of the most powerful patterns in AI workflows is the feedback loop, where one agent's output becomes another agent's input. This creates an iterative improvement cycle.

**The Feedback Process:**
1. **Initial Creation**: Comedian generates content
2. **Critical Analysis**: Critic evaluates the content  
3. **Reflection & Improvement**: Comedian receives feedback and refines their work

This pattern mimics real-world creative processes and often leads to significantly improved results compared to single-shot generation.

In [10]:
# messages for comedian again
prompt5 = "your jokes were evaluated by a critic, does it make sense to you? why?"
prompt6 = "basing on your reflection, write joke1 again"
prompt7 = "write joke2 again"

# let us provide critic's feedback to comedian
context2 = [i.response for i in critic.msgs.assistant_responses]

In [11]:
# run the instructions
reflect = await comedian.communicate(prompt5, context=context2)
joke11 = await comedian.communicate(prompt6)
joke22 = await comedian.communicate(prompt7)

In [12]:
Markdown(reflect)

Yes, it makes sense to me. The jokes rely on vivid oceanic imagery and clever puns—"whale out," "fin-ish," "fintastic"—which create a fun, playful mental picture. They work because they anthropomorphize marine creatures in an unexpected social setting, making the humor accessible and entertaining. The punchlines cleverly twist the situation, reinforcing the humor through wordplay. Given the ratings and the critic’s positive remarks about the visual and pun-based humor, it seems thoughtfully crafted to amuse an audience familiar with marine themes and puns.

In [13]:
Markdown(joke11)

Sure! Here's a refreshed version of Joke 1, keeping in line with the witty oceanic imagery and wordplay:

A blue whale and a big shark walk into a bar, spot each other, and start dancing. Turns out, one just wants to whale out, and the other’s ready to fin-ish the night with a splash!

In [14]:
Markdown(joke22)

# Assistant Response

**Response**:
And then they stopped—because even a giant whale and a massive shark knew it’s hard to keep dancing when the music’s just a little too *fintastic*.

In [15]:
comedian.to_df()

,created_at,role,content,id,sender,recipient,metadata
0,1.753196e+09,system,"{'system_message': 'As a comedian, you are sar...",3b518c53-5bbc-4d38-8cec-7601af2cda68,system,466fec09-0bfa-46bc-9c6d-5e00c6390ae6,{'lion_class': 'lionagi.protocols.messages.sys...
1,1.753196e+09,user,"{'context': [], 'instruction': 'short joke: a ...",58120383-0218-4cc2-b0d1-848020d660cf,user,466fec09-0bfa-46bc-9c6d-5e00c6390ae6,{'lion_class': 'lionagi.protocols.messages.ins...
2,1.753196e+09,assistant,{'assistant_response': 'A blue whale and a big...,e312733a-c22f-4996-9b0f-57c76127bbc4,466fec09-0bfa-46bc-9c6d-5e00c6390ae6,user,{'model_response': {'id': 'chatcmpl-Bw8kZu6vZc...
3,1.753196e+09,user,"{'context': [], 'instruction': 'continue the j...",c7699d50-20f8-4ea1-8ac6-afa68d8ae10e,user,466fec09-0bfa-46bc-9c6d-5e00c6390ae6,{'lion_class': 'lionagi.protocols.messages.ins...
4,1.753196e+09,assistant,{'assistant_response': 'And then they stopped....,602c39b2-436f-4027-8d2a-2db25930dcd7,466fec09-0bfa-46bc-9c6d-5e00c6390ae6,user,{'model_response': {'id': 'chatcmpl-Bw8ka5TX4t...
5,1.753196e+09,user,{'context': ['The first joke charmingly plays ...,510f6958-cb4e-428a-9cbb-c0cc100cd059,user,466fec09-0bfa-46bc-9c6d-5e00c6390ae6,{'lion_class': 'lionagi.protocols.messages.ins...
6,1.753196e+09,assistant,"{'assistant_response': 'Yes, it makes sense to...",f9383621-c009-4ece-843c-b64261703938,466fec09-0bfa-46bc-9c6d-5e00c6390ae6,user,{'model_response': {'id': 'chatcmpl-Bw8kfGGPtr...
7,1.753196e+09,user,"{'context': [], 'instruction': 'basing on your...",e580bf5e-8c97-450f-b4ff-efd5b2f85133,user,466fec09-0bfa-46bc-9c6d-5e00c6390ae6,{'lion_class': 'lionagi.protocols.messages.ins...
8,1.753196e+09,assistant,{'assistant_response': 'Sure! Here's a refresh...,4f09a1b2-2741-4859-a885-55c1ca835fa3,466fec09-0bfa-46bc-9c6d-5e00c6390ae6,user,{'model_response': {'id': 'chatcmpl-Bw8khpuvxO...
9,1.753196e+09,user,"{'context': [], 'instruction': 'write joke2 ag...",fa52197b-086b-446d-b77b-5bcc94768698,user,466fec09-0bfa-46bc-9c6d-5e00c6390ae6,{'lion_class': 'lionagi.protocols.messages.ins...


In [16]:
critic.to_df()

,created_at,role,content,id,sender,recipient,metadata
0,1.753196e+09,system,{'system_message': 'you are a respected commen...,efed13b2-5e13-418a-a282-6eeaa165f9eb,system,75e84403-697d-455f-82c1-34627e5a2edd,{'lion_class': 'lionagi.protocols.messages.sys...
1,1.753196e+09,user,{'context': [{'joke1': {'prompt': 'short joke:...,69721cd5-a17d-4e3b-8cb1-0d33a2e83725,user,75e84403-697d-455f-82c1-34627e5a2edd,{'lion_class': 'lionagi.protocols.messages.ins...
2,1.753196e+09,assistant,{'assistant_response': 'The first joke charmin...,a1f0d69b-d64a-40bb-a1e3-3632761e90c3,75e84403-697d-455f-82c1-34627e5a2edd,user,{'model_response': {'id': 'chatcmpl-Bw8kbOEO1c...
3,1.753196e+09,user,"{'context': [], 'instruction': 'provide a conc...",eecb954e-825f-43f8-b569-7251d403e69d,user,75e84403-697d-455f-82c1-34627e5a2edd,{'lion_class': 'lionagi.protocols.messages.ins...
4,1.753196e+09,assistant,{'assistant_response': 'The first joke creativ...,a0967acf-bce2-4b67-9817-00052b5d36ce,75e84403-697d-455f-82c1-34627e5a2edd,user,{'model_response': {'id': 'chatcmpl-Bw8kdNUI0r...


**Branch Persistence: Saving & Loading State**

One of lionagi's powerful features is the ability to serialize and persist entire conversation branches. This enables:

**Production Use Cases:**
- **Session Recovery**: Resume conversations after system restarts
- **Conversation Archiving**: Store important dialogues for future reference  
- **A/B Testing**: Save different conversation branches for comparison
- **Audit Trails**: Maintain complete records of AI interactions

The serialization captures not just the messages, but the entire conversation state including metadata, model responses, and conversation flow.

In [17]:
import json

with open("data/002_comedian.json", "w") as f:
    json.dump(comedian.to_dict(), f, indent=4)

with open("data/002_critic.json", "w") as f:
    json.dump(critic.to_dict(), f, indent=4)

In [18]:
# To use them again

with open("data/002_comedian.json") as f:
    dict_ = json.load(f)
    comedian = Branch.from_dict(dict_)

comedian.to_df()

,created_at,role,content,id,sender,recipient,metadata
0,1.753196e+09,system,{'system_message': {'id': '3b518c53-5bbc-4d38-...,e3946ed4-5fdf-4e25-a40c-8721c06b34bd,system,466fec09-0bfa-46bc-9c6d-5e00c6390ae6,{'lion_class': 'lionagi.protocols.messages.sys...
1,1.753196e+09,system,"{'system_message': 'As a comedian, you are sar...",3b518c53-5bbc-4d38-8cec-7601af2cda68,system,466fec09-0bfa-46bc-9c6d-5e00c6390ae6,{'lion_class': 'lionagi.protocols.messages.sys...
2,1.753196e+09,user,"{'context': [], 'instruction': 'short joke: a ...",58120383-0218-4cc2-b0d1-848020d660cf,user,466fec09-0bfa-46bc-9c6d-5e00c6390ae6,{'lion_class': 'lionagi.protocols.messages.ins...
3,1.753196e+09,assistant,{'assistant_response': 'A blue whale and a big...,e312733a-c22f-4996-9b0f-57c76127bbc4,466fec09-0bfa-46bc-9c6d-5e00c6390ae6,user,{'model_response': {'id': 'chatcmpl-Bw8kZu6vZc...
4,1.753196e+09,user,"{'context': [], 'instruction': 'continue the j...",c7699d50-20f8-4ea1-8ac6-afa68d8ae10e,user,466fec09-0bfa-46bc-9c6d-5e00c6390ae6,{'lion_class': 'lionagi.protocols.messages.ins...
5,1.753196e+09,assistant,{'assistant_response': 'And then they stopped....,602c39b2-436f-4027-8d2a-2db25930dcd7,466fec09-0bfa-46bc-9c6d-5e00c6390ae6,user,{'model_response': {'id': 'chatcmpl-Bw8ka5TX4t...
6,1.753196e+09,user,{'context': ['The first joke charmingly plays ...,510f6958-cb4e-428a-9cbb-c0cc100cd059,user,466fec09-0bfa-46bc-9c6d-5e00c6390ae6,{'lion_class': 'lionagi.protocols.messages.ins...
7,1.753196e+09,assistant,"{'assistant_response': 'Yes, it makes sense to...",f9383621-c009-4ece-843c-b64261703938,466fec09-0bfa-46bc-9c6d-5e00c6390ae6,user,{'model_response': {'id': 'chatcmpl-Bw8kfGGPtr...
8,1.753196e+09,user,"{'context': [], 'instruction': 'basing on your...",e580bf5e-8c97-450f-b4ff-efd5b2f85133,user,466fec09-0bfa-46bc-9c6d-5e00c6390ae6,{'lion_class': 'lionagi.protocols.messages.ins...
9,1.753196e+09,assistant,{'assistant_response': 'Sure! Here's a refresh...,4f09a1b2-2741-4859-a885-55c1ca835fa3,466fec09-0bfa-46bc-9c6d-5e00c6390ae6,user,{'model_response': {'id': 'chatcmpl-Bw8khpuvxO...
